In [1]:
import sys
sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

==================== Computational Cancer Analysis Library ====================
<11:03:44> Checking dependencies ...
<11:03:44> Using the following packages:
<11:03:44> 	matplotlib (v1.5.1)
<11:03:44> 	numpy (v1.10.4)
<11:03:44> 	pandas (v0.18.0)
<11:03:44> 	rpy2 (v2.7.9)
<11:03:44> 	scikit-learn (v0.17.1)
<11:03:44> 	scipy (v0.17.0)
<11:03:44> 	seaborn (v0.7.0)


# Make simulation matrix

In [ ]:
def add_value(df, inVal, outVal):
    # Add value in cluster
    # Set inVal or outVal to be None when not updating it
    
    for i,(n,s) in enumerate(df.iterrows()):
        #print('add_value:',i)
        for j,c in enumerate(s.index):
            if inVal and n==c:
                df.iloc[i,j]=inVal
            if outVal and n!=c:
                df.iloc[i,j]=outVal

def add_noise(df,inMu,inSigma,outMu,outSigma):
    # Add noise
    
    for i,(n,s) in enumerate(df.iterrows()):
        #print('add_noise:',i)
        for j,c in enumerate(s.index):
            if (inMu or inSigma) and n==c:
                df.iloc[i,j]+=random.gauss(inMu,inSigma)
            if (outMu or outSigma) and n!=c:
                df.iloc[i,j]+=random.gauss(outMu,outSigma)

def mix(df,k,mix):
    # Mix cluster values to nonclusters and vice versa
    
    assert k!=0, print('k cannot be 0')
    
    # Get the number of values in a cluster
    n_k_val=len(df.columns)*len(df.index)/k
    #print('n_k_val:',n_k_val)
    
    # Count
    c=0
    while c<mix*n_k_val:
        
        # Pick 1st random index and column
        r_idx0=random.randint(0,len(df.index)-1)
        r_col0=random.randint(0,len(df.columns)-1)

        # If index and column locate inside a cluster
        if df.index[r_idx0]==df.columns[r_col0]:    
            
            # Get cluster value located
            pick0=df.iloc[r_idx0,r_col0]
            
            # Pick 2nd random index and column
            r_idx1=random.randint(0,len(df.index)-1)
            r_col1=random.randint(0,len(df.columns)-1)
            
            # If index and column locate outside a cluster
            if df.index[r_idx1]!=df.columns[r_col1]:    

                # Get non-cluster value located
                pick1=df.iloc[r_idx1,r_col1]

                # Swap
                df.iloc[r_idx0,r_col0]=pick1
                df.iloc[r_idx1,r_col1]=pick0
                
                # Count
                c+=1
                
                #print('Swapped (%s,%s) & (%s,%s)'%(df.index[r_idx0],df.columns[r_col0],df.index[r_idx1],df.columns[r_col1]))

def initialize_simulation_df(df,inVal,inMu,inSigma,outVal,outMu,outSigma,mix):
    # Initialize values in and out of a cluster and add noise
    
    t0=time()
    
    # For each row
    for i,(n,s) in enumerate(df.iterrows()):
        print('initialize_simulation_df:',i)
        
        # For each column
        for j,c in enumerate(s.index):
            r=random.random()
            
            if mix and r<=mix:
                # Mix
                if n==c:
                    # In cluster gets out-value
                    df.iloc[i,j]=outVal
                    if outMu or outSigma:
                        df.iloc[i,j]+=random.gauss(outMu,outSigma)
                else:
                    # Out cluster gets in-value
                    df.iloc[i,j]=inVal
                    if inMu or inSigma:
                        df.iloc[i,j]+=random.gauss(inMu,inSigma) 
            else:
                # No mix
                if n==c:
                    # In cluster gets in-value
                    df.iloc[i,j]=inVal
                    if inMu or inSigma:
                        df.iloc[i,j]+=random.gauss(inMu,inSigma)
                else:
                    # Out cluster gets out-value
                    df.iloc[i,j]=outVal
                    if outMu or outSigma:
                        df.iloc[i,j]+=random.gauss(outMu,outSigma)
    print('initialize_simulation_df: done in %0.3fs.'%(time()-t0))

def plot_mtrx(mtrx):
    # Plot simulation matrix
    
    plt.imshow(mtrx,interpolation='nearest',cmap=plt.cm.ocean)
    plt.colorbar()
    plt.show()
    
def make_mtrx_sample_x_variable_simulation(n_sample,
                                           n_var,
                                           k,
                                           val_in,
                                           val_out,
                                           noise_in_mu=None,
                                           noise_in_sigma=None,
                                           noise_out_mu=None,
                                           noise_out_sigma=None,
                                           noise_mix=None,
                                           prefix_out_f=None,
                                           suffix_out_f=None,
                                           plot=False):
    """
    Make sample x variable matrix
    """
    assert k != 0, 'k cannot be 0'
    assert k <= n_var,'k cannot be greater than n_var'
    
    # Make an empty sample x variable matrix filled with 0
    mtrx_sample_x_var = pd.DataFrame(index=range(n_sample), columns=range(n_var)).fillna(0)

    # Slice dataframe index and column and make lists of dataframe indexes and columns for each index and column slice
    list_index_slice = toolK.slice_list(mtrx_sample_x_var.index, k)
    list_column_slice = toolK.slice_list(mtrx_sample_x_var.columns, k)

    # Make index and column slice x dataframe indexes dictionaries
    dict_index_slice = {}
    for i, l in enumerate(list_index_slice):
        dict_index_slice[i] = l
    dict_column_slice = {}
    for i, l in enumerate(list_column_slice):
        dict_column_slice[i] = l

    # Set dataframe index and column to be index and column slice indices respectively
    index=list(mtrx_sample_x_var.index)
    for i,l in dict_index_slice.items():
        for j in l:
            index[j]=i
    mtrx_sample_x_var.index=index
    columns=list(mtrx_sample_x_var.columns)
    for i,l in dict_column_slice.items():
        for j in l:
            columns[j]=i
    mtrx_sample_x_var.columns=columns

    # Initialize simulation matrix
    initialize_simulation_df(mtrx_sample_x_var,val_in,noise_in_mu,noise_in_sigma,val_out,noise_out_mu,noise_out_sigma,noise_mix)
    
    # Save
    if prefix_out_f:        
        mtrx_sample_x_var.to_csv(prefix_out_f+'_sample_%s_var_%s_k_%s_mix_%s_%s'%(n_sample,n_var,k,noise_mix,suffix_out_f),sep='\t')
    
    # Plot
    if plot:
        plot_mtrx(mtrx_sample_x_var)

# Make simulation matrix

# Set number of samples
list_n_sample = [100, 500, 1000, 5000]
# Set number of variables
list_n_var = [100, 500, 1000, 5000]
# Set values in clusters
val_in = 1
# Set values out of clusters
val_out = 0
# Set Ks
list_k = [1, 2, 3, 4, 5, 6, 10, 15, 20, 25]
# Set the fractions of cluster values to be swapped between clusters and nonclusters
list_noise_mix = [0, 0.05, 0.1, 0.2]
# Set noise in clusters
noise_in_mu = 0
noise_in_sigma = 0.1 * noise_in_mu
# Set noise out of clusters
noise_out_mu = 0
noise_out_sigma = 0.1*noise_out_mu

# Simulate
for sample in list_n_sample:
    print('sample:',sample)
    
    for var in list_n_var:
        print('\tvar:',var)
        
        for k in list_k:
            print('\t\tk:',k)
            
            for noise_mix in list_noise_mix:
                print('\t\t\tnoise_mix:',noise_mix)
                
                make_mtrx_sample_x_variable_simulation(sample,
                                                       var,
                                                       k,
                                                       val_in,
                                                       val_out,
                                                       noise_mix=noise_mix,
                                                       prefix_out_f='/cellar/users/hyeerna/aLL/mtrx_sample_x_var_simulation/test/',
                                                       suffix_out_f='sfx',
                                                       plot=True)